<a href="https://colab.research.google.com/github/hanxiang270/CASA0002/blob/main/W05_quiz_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 5 Quiz

This notebook contains the SQL Quiz for Week 5. Section 1 uses the New York City data we know and love. Section 2 will conduct a spatial damage assessment across the Gaza strip.

### INSTRUCTIONS:

Run this notebook in Google Colab. The answer to each quesiton will be a number or a string. Input these into the corresponding question on Moodle. You have 90 minutes to attempt the quiz, so if you get stuck on a question, move on.

Make sure you run all of the cells of code in order. If you run into serious problems, try clicking on the "runtime" tab above and selecting "restart session and run all".

# Section 1



This URL: `https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip` points to a .zip file containing shapefiles associated with the NYC data that we've been using in class so far. You can choose how to use sql, either the `con.sql()` syntax or the `%%sql` syntax. Either way, use the code cell below to install and import the necessary packages.

In [ ]:
%pip install duckdb leafmap

In [2]:
import duckdb
import leafmap
import pandas as pd

In [3]:
con = duckdb.connect()

con.install_extension("httpfs")
con.load_extension("httpfs")

con.install_extension("spatial")
con.load_extension("spatial")

In [4]:
url = "https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip"
leafmap.download_file(url, unzip=True)

postgis-workshop-2020.zip already exists. Skip downloading. Set overwrite=True to overwrite.


'/content/postgis-workshop-2020.zip'

In [8]:
import zipfile
import os

zip_path = "/content/postgis-workshop-2020.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()
print("Files inside the ZIP:", file_list)


Files inside the ZIP: ['postgis-workshop/', 'postgis-workshop/printing/', 'postgis-workshop/printing/nyc_data_dictionary.pdf', 'postgis-workshop/printing/postgis-workshop-exercises.docx', 'postgis-workshop/printing/postgis-workshop-exercises.pdf', 'postgis-workshop/printing/nyc_data_dictionary.docx', 'postgis-workshop/data/', 'postgis-workshop/data/nyc_homicides.dbf', 'postgis-workshop/data/nyc_neighborhoods.shx', 'postgis-workshop/data/nyc_neighborhoods.shp', 'postgis-workshop/data/nyc_census_blocks.shx', 'postgis-workshop/data/nyc_streets.prj', 'postgis-workshop/data/nyc_subway_stations.dbf', 'postgis-workshop/data/nyc_census_blocks.shp', 'postgis-workshop/data/nyc_subway_stations.shx', 'postgis-workshop/data/nyc_census_blocks.dbf', 'postgis-workshop/data/nyc_subway_stations.shp', 'postgis-workshop/data/nyc_neighborhoods.dbf', 'postgis-workshop/data/nyc_homicides.shx', 'postgis-workshop/data/nyc_census_sociodata.sql', 'postgis-workshop/data/2000/', 'postgis-workshop/data/2000/nyc_cen

## Question 1

Firslty, please provide the link to your colab notebook in the Moodle quiz using the following steps:

1. In the top right corner, click the "share" button
2. In the popup, click on "restricted" and change this to "anyone with the link"
3. Finally, click "copy link", and paste the link into the box for Question 1 on moodle.

Once you've done this, download and unzip the data, then create the following tables using the corresponding shapefiles.

- nyc_neighborhoods
- nyc_census_blocks
- nyc_homicides
- nyc_streets
- nyc_subway_stations

In [10]:
con.sql("""

CREATE TABLE nyc_neighborhoods AS SELECT * FROM ST_Read('postgis-workshop/data/nyc_neighborhoods.shp');
CREATE TABLE nyc_census_blocks AS SELECT * FROM ST_Read('postgis-workshop/data/nyc_census_blocks.shp');
CREATE TABLE nyc_homicides AS SELECT * FROM ST_Read('postgis-workshop/data/nyc_homicides.shp');
CREATE TABLE nyc_streets AS SELECT * FROM ST_Read('postgis-workshop/data/nyc_streets.shp');
CREATE TABLE nyc_subway_stations AS SELECT * FROM ST_Read('postgis-workshop/data/nyc_subway_stations.shp');


""")

## Question 2:
What is the longest street in New York? Note, it may be split up into multiple segments! Ignore missing and null values.


In [13]:
con.sql("""

SELECT name, ST_Length(geom)
FROM nyc_streets
ORDER BY ST_Length(geom)
DESC LIMIT 1;

""")

┌─────────────────┬────────────────────┐
│      NAME       │  st_length(geom)   │
│     varchar     │       double       │
├─────────────────┼────────────────────┤
│ Leif Ericson Dr │ 14918.122617388475 │
└─────────────────┴────────────────────┘

## Question 3:

Which borough had the fewest daytime shootings in 2009?

In [14]:
con.sql("""

PRAGMA table_info(nyc_homicides);

""")

┌───────┬────────────┬──────────┬─────────┬────────────┬─────────┐
│  cid  │    name    │   type   │ notnull │ dflt_value │   pk    │
│ int32 │  varchar   │ varchar  │ boolean │  varchar   │ boolean │
├───────┼────────────┼──────────┼─────────┼────────────┼─────────┤
│     0 │ INCIDENT_D │ DATE     │ false   │ NULL       │ false   │
│     1 │ BORONAME   │ VARCHAR  │ false   │ NULL       │ false   │
│     2 │ NUM_VICTIM │ VARCHAR  │ false   │ NULL       │ false   │
│     3 │ PRIMARY_MO │ VARCHAR  │ false   │ NULL       │ false   │
│     4 │ ID         │ BIGINT   │ false   │ NULL       │ false   │
│     5 │ WEAPON     │ VARCHAR  │ false   │ NULL       │ false   │
│     6 │ LIGHT_DARK │ VARCHAR  │ false   │ NULL       │ false   │
│     7 │ YEAR       │ BIGINT   │ false   │ NULL       │ false   │
│     8 │ geom       │ GEOMETRY │ false   │ NULL       │ false   │
└───────┴────────────┴──────────┴─────────┴────────────┴─────────┘

In [17]:
con.sql("""

FROM nyc_homicides;

""")

┌────────────┬───────────────┬────────────┬────────────┬───────┬─────────┬────────────┬───────┬──────────────────────────────────────────────┐
│ INCIDENT_D │   BORONAME    │ NUM_VICTIM │ PRIMARY_MO │  ID   │ WEAPON  │ LIGHT_DARK │ YEAR  │                     geom                     │
│    date    │    varchar    │  varchar   │  varchar   │ int64 │ varchar │  varchar   │ int64 │                   geometry                   │
├────────────┼───────────────┼────────────┼────────────┼───────┼─────────┼────────────┼───────┼──────────────────────────────────────────────┤
│ 2008-01-01 │ Brooklyn      │ 1          │ NULL       │     7 │ gun     │ D          │  2008 │ POINT (592158.6657641566 4502210.892367315)  │
│ 2008-01-04 │ Manhattan     │ 1          │ NULL       │    14 │ gun     │ D          │  2008 │ POINT (588654.9516122746 4517855.382656676)  │
│ 2008-01-05 │ Queens        │ 1          │ NULL       │    15 │ gun     │ D          │  2008 │ POINT (605800.8150245796 4505730.608395767)  │

In [18]:
con.sql("""

SELECT DISTINCT WEAPON
FROM nyc_homicides;

""")

┌──────────────────┐
│      WEAPON      │
│     varchar      │
├──────────────────┤
│ gun              │
│ knife            │
│ NULL             │
│ other            │
│ blunt_instrument │
└──────────────────┘

In [19]:
con.sql("""

SELECT BORONAME, COUNT(*) AS shooting_count
FROM nyc_homicides
WHERE YEAR = '2009'
AND "WEAPON" = 'gun'
AND "LIGHT_DARK" = 'D'
GROUP BY BORONAME
ORDER BY shooting_count ASC;

""")

┌───────────────┬────────────────┐
│   BORONAME    │ shooting_count │
│    varchar    │     int64      │
├───────────────┼────────────────┤
│ Staten Island │              6 │
│ Manhattan     │             12 │
│ Queens        │             18 │
│ The Bronx     │             44 │
│ Brooklyn      │             61 │
└───────────────┴────────────────┘

## Question 4:

What is the total population of the census blocks served by the L train?

In [20]:
con.sql("""

SELECT
SUM(popn_total) AS popn_total
FROM nyc_census_blocks AS c
JOIN nyc_subway_stations AS s
ON ST_DWithin(
  c.geom,
  s.geom,
  200
  )

WHERE strpos(s.routes,'L') > 0;

""")

┌────────────┐
│ popn_total │
│   int128   │
├────────────┤
│     110118 │
└────────────┘

## Question 5

which subway station had the largest number of homicides within 1km?

In [21]:
con.sql("""

SELECT s.name, COUNT(*) AS homicide_count
FROM nyc_homicides AS h
JOIN nyc_subway_stations AS s
ON ST_DWithin(
  h.geom,
  s.geom,
  1000
  )
GROUP BY s.name
ORDER BY homicide_count DESC;

""")

┌─────────────────┬────────────────┐
│      NAME       │ homicide_count │
│     varchar     │     int64      │
├─────────────────┼────────────────┤
│ 125th St        │            305 │
│ 116th St        │            261 │
│ Franklin Ave    │            251 │
│ 145th St        │            246 │
│ 170th St        │            214 │
│ Utica Ave       │            213 │
│ Sutter Ave      │            208 │
│ 135th St        │            208 │
│ Myrtle Ave      │            206 │
│ Broadway Jct    │            204 │
│      ·          │              · │
│      ·          │              · │
│      ·          │              · │
│ Broad Channel   │              1 │
│ Richmond Valley │              1 │
│ Tottenville     │              1 │
│ Annadale        │              1 │
│ Rockaway Park   │              1 │
│ Atlantic        │              1 │
│ Huguenot        │              1 │
│ Nassau          │              1 │
│ Oakwood Heights │              1 │
│ Prince's Bay    │              1 │
├

# Section 2

This section explores building damage in the Gaza Strip resulting from the ongoing war. You will conduct a geospatial analysis of building damage using two datasets:

1. [Humantiarian Open Street Map](https://www.hotosm.org/projects/gaza-building-footprints-pre-conflict-update-2024/) building footprints for the Gaza Strip
  * data: https://storage.googleapis.com/qm2/Gaza_Buildings_2.geojson.zip
2. [Damage points](https://unosat.org/products/3984) from the UN Satellite Agency (UNOSAT).
  * data: https://storage.googleapis.com/qm2/UNOSAT_GAZA_20240503_2.zip


The Coordinate Reference System of these datasets is EPSG:4326; therefore, stock functions like ST_DISTANCE() will yield values in degrees, not meters. Be mindful of this in your analysis, and use the DuckDB spatial functions [documentation](https://duckdb.org/docs/extensions/spatial/functions#st_geomfromtext) to your advantage.


First, download and unzip the data, and create two tables: `gaza_buildings`, containing the building footprint data, and `damage_points` containing the UNOSAT damage points. Make sure to set a spatial index on these tables, this will make your queries run much faster!

### **Table 1**

In [22]:
url = "https://storage.googleapis.com/qm2/Gaza_Buildings_2.geojson.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://storage.googleapis.com/qm2/Gaza_Buildings_2.geojson.zip
To: /content/Gaza_Buildings_2.geojson.zip
100%|██████████| 15.0M/15.0M [00:00<00:00, 57.2MB/s]


Extracting files...


'/content/Gaza_Buildings_2.geojson.zip'

In [23]:
con.sql("CREATE TABLE gaza_buildings AS SELECT * FROM ST_Read('Gaza_Buildings_2.geojson')")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [24]:
con.sql("""

CREATE INDEX gaza_buildings_geom_idx
ON gaza_buildings USING Rtree (geom);

""")

### **Table 2**

In [25]:
url2 = "https://storage.googleapis.com/qm2/UNOSAT_GAZA_20240503_2.zip"
leafmap.download_file(url2, unzip=True)

Downloading...
From: https://storage.googleapis.com/qm2/UNOSAT_GAZA_20240503_2.zip
To: /content/UNOSAT_GAZA_20240503_2.zip
100%|██████████| 4.98M/4.98M [00:00<00:00, 141MB/s]


Extracting files...


'/content/UNOSAT_GAZA_20240503_2.zip'

In [27]:
zip_path_2 = "/content/UNOSAT_GAZA_20240503_2.zip"

with zipfile.ZipFile(zip_path_2, 'r') as zip_ref:
    file_list = zip_ref.namelist()
print("Files inside the ZIP:", file_list)

Files inside the ZIP: ['UNOSAT_GAZA_20240503_2.shp', 'UNOSAT_GAZA_20240503_2.prj', 'UNOSAT_GAZA_20240503_2.cpg', 'UNOSAT_GAZA_20240503_2.dbf', 'UNOSAT_GAZA_20240503_2.shx']


In [28]:
con.sql("""

CREATE TABLE damage_points AS SELECT * FROM ST_Read('UNOSAT_GAZA_20240503_2.shp');

""")

In [29]:
con.sql("""

CREATE INDEX damage_points_geom_idx
ON damage_points USING Rtree (geom);

""")

## Question 6

Create a table called "gaza_buildings_damaged" which is comprised only of building footprints that intersect with a UNOSAT damage point. How many damaged buildings are there in Gaza?

In [31]:
con.sql("""

PRAGMA table_info(gaza_buildings);

""")

┌───────┬──────────┬──────────┬─────────┬────────────┬─────────┐
│  cid  │   name   │   type   │ notnull │ dflt_value │   pk    │
│ int32 │ varchar  │ varchar  │ boolean │  varchar   │ boolean │
├───────┼──────────┼──────────┼─────────┼────────────┼─────────┤
│     0 │ osm_id   │ INTEGER  │ false   │ NULL       │ false   │
│     1 │ osm_type │ VARCHAR  │ false   │ NULL       │ false   │
│     2 │ building │ VARCHAR  │ false   │ NULL       │ false   │
│     3 │ name     │ VARCHAR  │ false   │ NULL       │ false   │
│     4 │ geom     │ GEOMETRY │ false   │ NULL       │ false   │
└───────┴──────────┴──────────┴─────────┴────────────┴─────────┘

In [37]:
con.sql("""

SELECT DISTINCT building
FROM gaza_buildings;

""")

┌─────────────────────────┐
│        building         │
│         varchar         │
├─────────────────────────┤
│ mosque                  │
│ residential             │
│ commercial              │
│ house                   │
│ school                  │
│ roof                    │
│ destroyed               │
│ garage                  │
│ college                 │
│ retail                  │
│   ·                     │
│   ·                     │
│   ·                     │
│ hangar                  │
│ greengrocer             │
│ hotel                   │
│ farm                    │
│ religious               │
│ yes;commercial          │
│ greenhouse_horticulture │
│ yes;hospital            │
│ water_tower             │
│ G                       │
├─────────────────────────┤
│   44 rows (20 shown)    │
└─────────────────────────┘

In [32]:
con.sql("""

PRAGMA table_info(damage_points);

""")

┌───────┬────────────┬──────────┬─────────┬────────────┬─────────┐
│  cid  │    name    │   type   │ notnull │ dflt_value │   pk    │
│ int32 │  varchar   │ varchar  │ boolean │  varchar   │ boolean │
├───────┼────────────┼──────────┼─────────┼────────────┼─────────┤
│     0 │ OBJECTID   │ DOUBLE   │ false   │ NULL       │ false   │
│     1 │ SiteID     │ BIGINT   │ false   │ NULL       │ false   │
│     2 │ SensorDate │ VARCHAR  │ false   │ NULL       │ false   │
│     3 │ SensorID   │ BIGINT   │ false   │ NULL       │ false   │
│     4 │ Confidence │ BIGINT   │ false   │ NULL       │ false   │
│     5 │ Main_Damag │ BIGINT   │ false   │ NULL       │ false   │
│     6 │ SensorDa_1 │ VARCHAR  │ false   │ NULL       │ false   │
│     7 │ SensorID_2 │ BIGINT   │ false   │ NULL       │ false   │
│     8 │ Confiden_1 │ BIGINT   │ false   │ NULL       │ false   │
│     9 │ Main_Dam_1 │ BIGINT   │ false   │ NULL       │ false   │
│     · │     ·      │   ·      │   ·     │  ·         │   ·  

In [45]:
con.sql("""

CREATE TABLE gaza_buildings_damaged_2 AS
SELECT *
FROM gaza_buildings AS b
JOIN damage_points AS d
ON ST_Intersects(b.geom, d.geom);

""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [46]:
con.sql("""
SELECT COUNT(*)
FROM gaza_buildings_damaged_2;

""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       109495 │
└──────────────┘

## Question 7

What is the total area (in square kilometers) of damaged buildings in Gaza? note: there are TWO area functions in duckDB, and only one of them returns answers in meters.

In [49]:
con.sql("""

SELECT SUM(ST_Area_Spheroid(geom)) / 1e6 AS total_area_km2
FROM gaza_buildings_damaged_2;


""")

┌────────────────────┐
│   total_area_km2   │
│       double       │
├────────────────────┤
│ 22.571649256731085 │
└────────────────────┘

## Question 8

What percentage of hospitals in Gaza have been damaged?


In [39]:
con.sql("""

SELECT COUNT(*) AS num_hospitals
FROM gaza_buildings
WHERE building = 'hospital';

""")

┌───────────────┐
│ num_hospitals │
│     int64     │
├───────────────┤
│            52 │
└───────────────┘

In [40]:
con.sql("""

SELECT COUNT(*) AS num_damaged_hospitals
FROM gaza_buildings AS b
JOIN damage_points AS d
ON ST_Intersects(b.geom, d.geom)
WHERE b.building = 'hospital';

""")

┌───────────────────────┐
│ num_damaged_hospitals │
│         int64         │
├───────────────────────┤
│                    34 │
└───────────────────────┘

## Question 9

The UNOSAT data contains a column called "Governorat" (shapefiles only allow column names of up to 10 characters). Calculate the number of damaged schools in each governorate.

In [51]:
con.sql("""

SELECT Governorat, COUNT(*) AS num_damaged_schools
FROM gaza_buildings AS b
JOIN damage_points AS d
ON ST_Intersects(b.geom, d.geom)
WHERE b.building = 'school'
GROUP BY "Governorat";

""")

┌───────────────┬─────────────────────┐
│  Governorat   │ num_damaged_schools │
│    varchar    │        int64        │
├───────────────┼─────────────────────┤
│ Gaza          │                  69 │
│ North Gaza    │                  29 │
│ Khan Yunis    │                  46 │
│ Deir Al-Balah │                   6 │
└───────────────┴─────────────────────┘

## Question 10

The "Dar Al-Shifa Hospital" was the largest medical complex in the gaza strip. Find the hospital's building footprint, and calculate the proportion of damaged buildings within 1 kilometer.

In [54]:
con.sql("""

SELECT name,ST_AsText(geom)
FROM gaza_buildings
WHERE name = 'Dar Al-Shifa Hospital';

""")

┌───────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│         name          │                                                                                                                                                                                                                                      

In [57]:
con.sql("""

SELECT COUNT(*)
FROM gaza_buildings
WHERE ST_DWithin(geom, 'POLYGON ((34.4431088 31.5238996, 34.4433649 31.5237154, 34.4434647 31.5236754, 34.4435236 31.5237113, 34.4435691 31.523728, 34.443532 31.5238226, 34.4435147 31.5238241, 34.4435143 31.5238788, 34.4435033 31.5238903, 34.4435607 31.5239425, 34.4436761 31.5239248, 34.4437785 31.5240182, 34.4436617 31.5241291, 34.443741 31.5241361, 34.443786 31.52414, 34.4439107 31.5240469, 34.4439286 31.5240603, 34.4441548 31.5238869, 34.4441709 31.5239021, 34.4441915 31.5238862, 34.4441733 31.523869, 34.4442764 31.5237898, 34.4445204 31.5240205, 34.4440399 31.5243898, 34.4438119 31.5241742, 34.4437809 31.5241947, 34.4437621 31.5241762, 34.4437143 31.5241712, 34.4436223 31.5241615, 34.4435023 31.5242574, 34.4431088 31.5238996))',1000)


""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       335184 │
└──────────────┘

In [58]:
con.sql("""

SELECT COUNT(*)
FROM gaza_buildings
WHERE ST_DWithin(
    ST_Transform(geom, 3857),  -- Convert building geometries to meters
    ST_Transform(ST_GeomFromText(
        'POLYGON ((34.4431088 31.5238996, 34.4433649 31.5237154, 34.4434647 31.5236754,
        34.4435236 31.5237113, 34.4435691 31.523728, 34.443532 31.5238226,
        34.4435147 31.5238241, 34.4435143 31.5238788, 34.4435033 31.5238903,
        34.4435607 31.5239425, 34.4436761 31.5239248, 34.4437785 31.5240182,
        34.4436617 31.5241291, 34.443741 31.5241361, 34.443786 31.52414,
        34.4439107 31.5240469, 34.4439286 31.5240603, 34.4441548 31.5238869,
        34.4441709 31.5239021, 34.4441915 31.5238862, 34.4441733 31.523869,
        34.4442764 31.5237898, 34.4445204 31.5240205, 34.4440399 31.5243898,
        34.4438119 31.5241742, 34.4437809 31.5241947, 34.4437621 31.5241762,
        34.4437143 31.5241712, 34.4436223 31.5241615, 34.4435023 31.5242574,
        34.4431088 31.5238996))', 4326), 3857),  -- Convert polygon to meters
    1000  -- Now interpreted correctly as meters
);

""")


BinderException: Binder Error: No function matches the given name and argument types 'ST_Transform(GEOMETRY, INTEGER_LITERAL)'. You might need to add explicit type casts.
	Candidate functions:
	ST_Transform(BOX_2D, VARCHAR, VARCHAR) -> BOX_2D
	ST_Transform(BOX_2D, VARCHAR, VARCHAR, BOOLEAN) -> BOX_2D
	ST_Transform(POINT_2D, VARCHAR, VARCHAR) -> POINT_2D
	ST_Transform(POINT_2D, VARCHAR, VARCHAR, BOOLEAN) -> POINT_2D
	ST_Transform(GEOMETRY, VARCHAR, VARCHAR) -> GEOMETRY
	ST_Transform(GEOMETRY, VARCHAR, VARCHAR, BOOLEAN) -> GEOMETRY
